# Introduction

- In this Notebook we will build and test a simple RAG System using LangChain and Ollama.
- Is the same steps as the **RAG_1** , but here we are using documents.

# Dependencies :

In [ ]:
!pip install -qU langchain langchain_community
!pip install -qU langchain_chroma
!pip install -qU langchain_ollama
!pip install pypdf

- To work with Ollama on Colab, we have an additional library, so we can launch a terminal just like in our local machine !

1. `pciutils` is required by Ollama to detect the GPU type.
2. Installation of Ollama in the runtime instance will be taken care by `curl -fsSL https://ollama.com/install.sh | sh`




In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (463 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package pci.ids.
(Reading database ... 123630 files and directories currently installed.)
Preparing to unpack .../pci.ids_0.0~2022.01.22-1_all.deb ...
Unpacking pci.ids (0.0~2022.01.22-1) ...
Selecting previously

# Implimentation :

## Running Ollama
---

In order to use Ollama it needs to run as a service in background parallel to your scripts. Becasue Jupyter Notebooks is built to run code blocks in sequence this make it difficult to run two blocks at the same time. As a workaround we will create a service using subprocess in Python so it doesn't block any cell from running.

Service can be started by command `ollama serve`.

`time.sleep(5)` adds some delay to get the Ollama service up before downloading the model.

In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

## Pulling Model
---

Download the LLM model using `ollama pull llama3.2`.

For other models check https://ollama.com/library

In [ ]:
!ollama pull llama3.1:8b

In [ ]:
!ollama list

NAME           ID              SIZE      MODIFIED               
llama3.1:8b    42182419e950    4.7 GB    Less than a second ago    


## Load LLM and Embedding Model

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub
import os


In [ ]:
model_name = "llama3.1:8b"

In [ ]:
llm = Ollama(model=model_name,base_url = "http://localhost:11434")
embed_model = OllamaEmbeddings(
    model=model_name,
    base_url="http://localhost:11434"
)

## Load Docs from Github

- I put the Documents in a Github repo , so you Sir didn't find any **Path issues**

In [1]:
!git clone https://github.com/NourTechNerd/data.git

Cloning into 'data'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 407.19 KiB | 3.34 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
pdf_paths =[
    "/content/data/Noureddine_Saidi.pdf",
    "/content/data/Cover letter.pdf"
]


## Chunking

In [ ]:
documents = []
for pdf_path in pdf_paths:
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"File {pdf_path} not found")
    loader = PyPDFLoader(pdf_path)
    doc = loader.load()
    documents.extend(doc)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600,chunk_overlap=200)
docs = text_splitter.split_documents(documents)

In [ ]:
print(f"Number of Chunks = {len(docs)}")
print(f"lenght of a first Chunk = {len(docs[0].page_content)}")
print("#############################")
print("Content of the first Chunk :",docs[0].page_content)


Number of Chunks = 14
lenght of a first Chunk = 598
#############################
Content of the first Chunk : CERTIFICATIONS
CONTACT
noureddinesaidi111@gmail.com
IMM D8, Basma 3, Riad Zitoune, Meknes
https://noureddinesaidi.netlify.app/
SKILLS
Project Management
Public Relations
Teamwork
Time Management
Leadership
Effective Communication
Critical Thinking
COURSERA
Machine Learning Specialization 
2023- 2024
English (B2)
French (C1)
LANGUAGES
EXPERIENCE
EDUCATION
SUMMARY
Passionate student with enthusiasm in Artificial Intelligence, Generative AI,
Machine Learning. Motivated by the exploration and application of state-of-the-art
GenAI technologies to address intricate challenges and develop inventive


## Create Database

In [ ]:
persist_directory = "CHROMA"
vector_store = Chroma.from_documents(docs,embed_model,persist_directory=persist_directory)


## The Retriever

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k":3})

In [ ]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
retrieval_qa_chat_prompt

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMes

## Chain

In [ ]:
# Create the Chain
combined_docs_chain = create_stuff_documents_chain(llm,retrieval_qa_chat_prompt)
retrieval_chain = create_retrieval_chain(retriever,combined_docs_chain)

## Test

- If it takes a lot of time with T4, it is probably a problem with Ollama Server, you need just the repeat the steps in **dependencies** section.

In [ ]:
question = "What are Skills  and Competences of Noureddine"
response = retrieval_chain.invoke({"input":question})
response

{'input': 'What are Skills  and Competences of Noureddine',
 'context': [Document(metadata={'page': 0, 'source': '/content/data/Cover letter.pdf'}, page_content='innovation. Thank you for considering my application. I would welcome the opportunity to discuss how \nmy skills and aspirations align with the objectives of this internship. \n \n \nSincerely, \nNoureddine Saidi'),
  Document(metadata={'page': 0, 'source': '/content/data/Noureddine_Saidi.pdf'}, page_content='automate weekly project reporting by integrating data from Jira, Microsoft\nTeams, and emails.  \nDeveloped comprehensive reports detailing completed tasks, ongoing tasks,\nnext week’s plan, and potential risks, enhancing project visibility and\naccountability.  \nDeployed the system on the JESA server using Docker, leveraging cloud\ncomputing for scalable and efficient data processing.  \nStreamlined the reporting process by reducing manual effort and utilizing APIs\nfor seamless data integration, resulting in significan

In [ ]:
print(response["answer"])

Based on the context, the skills and competences of Noureddine Saidi appear to be:

1. **Technical skills**:
	* Proficiency in Jira and Microsoft Teams
	* Experience with email integration
	* Knowledge of Docker and cloud computing for scalable data processing
	* Familiarity with APIs for seamless data integration
2. **Analytical skills**:
	* Ability to develop comprehensive reports from multiple data sources (Jira, Teams, emails)
	* Capacity to identify key metrics (completed tasks, ongoing tasks, next week's plan, potential risks)
3. **Project management skills**:
	* Experience in managing project reporting and visibility
	* Understanding of accountability and risk management in projects
4. **Soft skills**:
	* Effective communication skills (as demonstrated by his ability to write reports and communicate with team members)
	* Adaptability and proactivity in tackling challenges
	* Ability to work towards shared goals and contribute to a team's success


## Export Your Database

In [ ]:
!zip -r /content/CHROMA.zip /content/CHROMA


  adding: content/CHROMA/ (stored 0%)
  adding: content/CHROMA/6c01c9e6-471f-414d-81ea-8af680c819aa/ (stored 0%)
  adding: content/CHROMA/6c01c9e6-471f-414d-81ea-8af680c819aa/header.bin (deflated 61%)
  adding: content/CHROMA/6c01c9e6-471f-414d-81ea-8af680c819aa/data_level0.bin (deflated 100%)
  adding: content/CHROMA/6c01c9e6-471f-414d-81ea-8af680c819aa/length.bin (deflated 54%)
  adding: content/CHROMA/6c01c9e6-471f-414d-81ea-8af680c819aa/link_lists.bin (stored 0%)
  adding: content/CHROMA/chroma.sqlite3 (deflated 47%)


In [ ]:
from google.colab import files
files.download('/content/CHROMA.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>